<a href="https://colab.research.google.com/github/ShinAsakawa/2019seminar_info/blob/master/notebooks/2019karpathys_pg_pong.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# Deep Reinforcement Learning: Pong from Pixels

## 方策勾配法 (Policy gradient) を用いた強化学習エージェントの学習 (OpenAI gym)

- [Original `pg-pong.py`](https://gist.github.com/karpathy/a4166c7fe253700972fcbc77e4ea32c5)
- このファイルは Andrej Karpathy の [ブログ記事](http://karpathy.github.io/2016/05/31/rl/) にあった `pg-pong.py` を Python3 で動くように修正し，かつ colaboratory で実行できるよう修正したものです。優れたブログ記事を書いた Karpathy に敬意を評します。

---

<center>
<img src="http://karpathy.github.io/assets/rl/preview.jpeg" style="width:39%;"><br>
<strong>市井の強化学習。左からアタリのゲームをするディープ Q ネット，アルファ碁，バークレイロボットによるレゴ積み木，四足歩行の物理シミュレーション Karpathy のブログより</strong>
</center>
    
ちなみに Karpathy はこのブログの中で，最近の進歩は次の 4 点が大きいと書いています。

1. 計算  (the obvious one: Moore’s Law, GPUs, ASICs),
1. データ (in a nice form, not just out there somewhere on the internet - e.g. ImageNet),
1. アルゴリズム (research and ideas, e.g. backprop, CNN, LSTM), and
1. インフラ (software under you - Linux, TCP/IP, Git, ROS, PR2, AWS, AMT, TensorFlow, etc.).
---

<center>
    <img src="http://karpathy.github.io/assets/rl/policy.png" style="width:39%"><br>
    <strong>2層のポリシーネットワーク</strong>
</center>

---

次の2つの図は通常の教師ありニューラルネットワークと強化学習との違いを表しています。

<center>
    <img src="http://karpathy.github.io/assets/rl/sl.png" style="width:74%"><br>
    <strong>通常のニューラルネットワーク</strong>
</center>

<center>
    <img src="http://karpathy.github.io/assets/rl/rl.png" style="width:74%"><br>
    <strong>強化学習</strong>
</center>


\begin{align}
\nabla_{\theta} E_x[f(x)] &= \nabla_{\theta} \sum_x p(x) f(x) & \text{definition of expectation} \\
& = \sum_x \nabla_{\theta} p(x) f(x) & \text{swap sum and gradient} \\
& = \sum_x p(x) \frac{\nabla_{\theta} p(x)}{p(x)} f(x) & \text{both multiply and divide by } p(x) \\
& = \sum_x p(x) \nabla_{\theta} \log p(x) f(x) & \text{use the fact that } \nabla_{\theta} \log(z) = \frac{1}{z} \nabla_{\theta} z \\
& = E_x[f(x) \nabla_{\theta} \log p(x) ] & \text{definition of expectation}
\end{align}


In [0]:
""" Trains an agent with (stochastic) Policy Gradients on Pong. Uses OpenAI Gym. """
from __future__ import print_function
from six.moves import xrange
from six.moves import cPickle

' Trains an agent with (stochastic) Policy Gradients on Pong. Uses OpenAI Gym. '

In [0]:
import numpy as np
#import cPickle as pickle
import pickle
import gym

In [0]:
# hyperparameters
H = 200 # number of hidden layer neurons
batch_size = 10 # every how many episodes to do a param update?
learning_rate = 1e-4
gamma = 0.99 # discount factor for reward
decay_rate = 0.99 # decay factor for RMSProp leaky sum of grad^2
resume = False # resume from previous checkpoint?
render = False

In [0]:
# model initialization
D = 80 * 80 # input dimensionality: 80x80 grid
if resume:
    model = pickle.load(open('save.p', 'rb'))
else:
    model = {}
    model['W1'] = np.random.randn(H,D) / np.sqrt(D) # "Xavier" initialization
    model['W2'] = np.random.randn(H) / np.sqrt(H)

In [0]:
#update buffers that add up gradients over a batch
#grad_buffer = { k : np.zeros_like(v) for k,v in model.iteritems() } 
grad_buffer = {k:np.zeros_like(v) for k,v in model.items()}     # python 3 対応のため書き換え

#rmsprop memory
#rmsprop_cache = { k : np.zeros_like(v) for k,v in model.iteritems() }
rmsprop_cache = {k:np.zeros_like(v) for k,v in model.items()}

In [0]:
def sigmoid(x):
    return 1.0 / (1.0 + np.exp(-x)) # sigmoid "squashing" function to interval [0,1]

def prepro(I):
    """ prepro 210x160x3 uint8 frame into 6400 (80x80) 1D float vector """
    I = I[35:195] # crop
    I = I[::2,::2,0] # downsample by factor of 2
    I[I == 144] = 0 # erase background (background type 1)
    I[I == 109] = 0 # erase background (background type 2)
    I[I != 0] = 1 # everything else (paddles, ball) just set to 1
    return I.astype(np.float).ravel()

In [0]:
def discount_rewards(r):
    """ take 1D float array of rewards and compute discounted reward """
    discounted_r = np.zeros_like(r)
    running_add = 0
    for t in reversed(xrange(0, r.size)):
        if r[t] != 0: running_add = 0 # reset the sum, since this was a game boundary (pong specific!)
        running_add = running_add * gamma + r[t]
        discounted_r[t] = running_add
    return discounted_r

In [0]:
def policy_forward(x):
    h = np.dot(model['W1'], x)
    h[h<0] = 0 # ReLU nonlinearity
    logp = np.dot(model['W2'], h)
    p = sigmoid(logp)
    return p, h # return probability of taking action 2, and hidden state

In [0]:
def policy_backward(eph, epdlogp):
    """ backward pass. (eph is array of intermediate hidden states) """
    dW2 = np.dot(eph.T, epdlogp).ravel()
    dh = np.outer(epdlogp, model['W2'])
    dh[eph <= 0] = 0 # backpro prelu
    dW1 = np.dot(dh.T, epx)
    return {'W1':dW1, 'W2':dW2}

In [0]:
env = gym.make("Pong-v0")
observation = env.reset()
prev_x = None # used in computing the difference frame
xs,hs,dlogps,drs = [],[],[],[]
running_reward = None
reward_sum = 0
episode_number = 0

---

<font size="+2" color="red">注:以下を実行すると無限ループです。適当なところで止めてください</font>


In [0]:
while True:
    if render: 
        env.render()

    # preprocess the observation, set input to network to be difference image
    cur_x = prepro(observation)
    x = cur_x - prev_x if prev_x is not None else np.zeros(D)
    prev_x = cur_x

    # forward the policy network and sample an action from the returned probability
    aprob, h = policy_forward(x)
    action = 2 if np.random.uniform() < aprob else 3 # roll the dice!

    # record various intermediates (needed later for backprop)
    xs.append(x) # observation
    hs.append(h) # hidden state
    y = 1 if action == 2 else 0 # a "fake label"
    dlogps.append(y - aprob) 
    # grad that encourages the action that was taken to be taken
    #(see http://cs231n.github.io/neural-networks-2/#losses if confused)

    # step the environment and get new measurements
    observation, reward, done, info = env.step(action)
    reward_sum += reward

    drs.append(reward) # record reward (has to be done after we call step() 
                       #  to get reward for previous action)

    if done: # an episode finished
        episode_number += 1

        # stack together all inputs, hidden states, action gradients, and rewards for this episode
        epx = np.vstack(xs)
        eph = np.vstack(hs)
        epdlogp = np.vstack(dlogps)
        epr = np.vstack(drs)
        xs,hs,dlogps,drs = [],[],[],[] # reset array memory

        # compute the discounted reward backwards through time
        discounted_epr = discount_rewards(epr)
        # standardize the rewards to be unit normal (helps control the gradient estimator variance)
        discounted_epr -= np.mean(discounted_epr)
        discounted_epr /= np.std(discounted_epr)

        epdlogp *= discounted_epr # modulate the gradient with advantage (PG magic happens right here.)
        grad = policy_backward(eph, epdlogp)
        for k in model: grad_buffer[k] += grad[k] # accumulate grad over batch

        # perform rmsprop parameter update every batch_size episodes
        if episode_number % batch_size == 0:
            for k,v in model.items():
                g = grad_buffer[k] # gradient
                rmsprop_cache[k] = decay_rate * rmsprop_cache[k] + (1 - decay_rate) * g**2
                model[k] += learning_rate * g / (np.sqrt(rmsprop_cache[k]) + 1e-5)
                grad_buffer[k] = np.zeros_like(v) # reset batch gradient buffer

        # boring book-keeping
        running_reward = reward_sum if running_reward is None else running_reward * 0.99 + reward_sum * 0.01
        print('resetting env. episode reward total was %f. running mean: %f' % (reward_sum, running_reward))
        if episode_number % 100 == 0: pickle.dump(model, open('save.p', 'wb'))
        reward_sum = 0
        observation = env.reset() # reset env
        prev_x = None

    if reward != 0: # Pong has either +1 or -1 reward exactly when game ends.
        print('ep %d: game finished, reward: %f' % (episode_number, reward), end='')
        print('') if reward == -1 else ' !!!!!!!!'

ep 0: game finished, reward: -1.000000
ep 0: game finished, reward: -1.000000
ep 0: game finished, reward: -1.000000
ep 0: game finished, reward: -1.000000
ep 0: game finished, reward: -1.000000
ep 0: game finished, reward: -1.000000
ep 0: game finished, reward: -1.000000
ep 0: game finished, reward: -1.000000
ep 0: game finished, reward: -1.000000
ep 0: game finished, reward: -1.000000
ep 0: game finished, reward: -1.000000
ep 0: game finished, reward: -1.000000
ep 0: game finished, reward: -1.000000
ep 0: game finished, reward: -1.000000
ep 0: game finished, reward: -1.000000
ep 0: game finished, reward: -1.000000
ep 0: game finished, reward: -1.000000
ep 0: game finished, reward: -1.000000
ep 0: game finished, reward: -1.000000
ep 0: game finished, reward: -1.000000
resetting env. episode reward total was -21.000000. running mean: -21.000000
ep 1: game finished, reward: -1.000000
ep 1: game finished, reward: -1.000000
ep 1: game finished, reward: -1.000000
ep 1: game finished, rewar

' !!!!!!!!'

ep 1: game finished, reward: -1.000000
ep 1: game finished, reward: -1.000000
ep 1: game finished, reward: -1.000000
ep 1: game finished, reward: -1.000000
ep 1: game finished, reward: -1.000000
ep 1: game finished, reward: -1.000000
ep 1: game finished, reward: -1.000000
ep 1: game finished, reward: -1.000000
ep 1: game finished, reward: -1.000000
ep 1: game finished, reward: -1.000000
ep 1: game finished, reward: -1.000000
ep 1: game finished, reward: -1.000000
ep 1: game finished, reward: -1.000000
ep 1: game finished, reward: -1.000000
ep 1: game finished, reward: -1.000000
ep 1: game finished, reward: -1.000000
ep 1: game finished, reward: -1.000000
resetting env. episode reward total was -20.000000. running mean: -20.990000
ep 2: game finished, reward: -1.000000
ep 2: game finished, reward: -1.000000
ep 2: game finished, reward: -1.000000
ep 2: game finished, reward: -1.000000
ep 2: game finished, reward: -1.000000
ep 2: game finished, reward: -1.000000
ep 2: game finished, rewar

' !!!!!!!!'

ep 2: game finished, reward: -1.000000
ep 2: game finished, reward: -1.000000
ep 2: game finished, reward: -1.000000
ep 2: game finished, reward: -1.000000
ep 2: game finished, reward: -1.000000
ep 2: game finished, reward: -1.000000
ep 2: game finished, reward: -1.000000
resetting env. episode reward total was -20.000000. running mean: -20.980100
ep 3: game finished, reward: -1.000000
ep 3: game finished, reward: -1.000000
ep 3: game finished, reward: -1.000000
ep 3: game finished, reward: -1.000000
ep 3: game finished, reward: -1.000000
ep 3: game finished, reward: -1.000000
ep 3: game finished, reward: -1.000000
ep 3: game finished, reward: -1.000000
ep 3: game finished, reward: -1.000000
ep 3: game finished, reward: -1.000000
ep 3: game finished, reward: -1.000000
ep 3: game finished, reward: -1.000000
ep 3: game finished, reward: -1.000000
ep 3: game finished, reward: -1.000000
ep 3: game finished, reward: -1.000000
ep 3: game finished, reward: -1.000000
ep 3: game finished, rewar

...

' !!!!!!!!'

ep 257: game finished, reward: -1.000000
ep 257: game finished, reward: -1.000000
ep 257: game finished, reward: -1.000000
ep 257: game finished, reward: -1.000000
ep 257: game finished, reward: -1.000000
ep 257: game finished, reward: -1.000000
ep 257: game finished, reward: -1.000000
ep 257: game finished, reward: -1.000000
ep 257: game finished, reward: -1.000000
resetting env. episode reward total was -20.000000. running mean: -20.437127
ep 258: game finished, reward: -1.000000
ep 258: game finished, reward: -1.000000
ep 258: game finished, reward: -1.000000
ep 258: game finished, reward: -1.000000
ep 258: game finished, reward: -1.000000
ep 258: game finished, reward: -1.000000
ep 258: game finished, reward: -1.000000
ep 258: game finished, reward: -1.000000
ep 258: game finished, reward: -1.000000
ep 258: game finished, reward: -1.000000
ep 258: game finished, reward: -1.000000
ep 258: game finished, reward: -1.000000
ep 258: game finished, reward: -1.000000
ep 258: game finished

' !!!!!!!!'

ep 259: game finished, reward: -1.000000
ep 259: game finished, reward: -1.000000
ep 259: game finished, reward: -1.000000
resetting env. episode reward total was -20.000000. running mean: -20.438328
ep 260: game finished, reward: -1.000000
ep 260: game finished, reward: -1.000000
ep 260: game finished, reward: -1.000000
ep 260: game finished, reward: -1.000000
ep 260: game finished, reward: -1.000000
ep 260: game finished, reward: -1.000000
ep 260: game finished, reward: -1.000000
ep 260: game finished, reward: -1.000000
ep 260: game finished, reward: -1.000000
ep 260: game finished, reward: -1.000000
ep 260: game finished, reward: -1.000000
ep 260: game finished, reward: -1.000000
ep 260: game finished, reward: -1.000000
ep 260: game finished, reward: -1.000000
ep 260: game finished, reward: -1.000000
ep 260: game finished, reward: -1.000000
ep 260: game finished, reward: -1.000000
ep 260: game finished, reward: -1.000000
ep 260: game finished, reward: -1.000000
ep 260: game finished

KeyboardInterrupt: 